<a href="https://colab.research.google.com/github/LarsGart/NLP-Social-Media-Mining-for-Health-Monitoring/blob/experimental/Task%202/Task_2_MedDRA_EMB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%%capture
!pip install demoji
!pip install transformers 
!pip install pytorch-lightning
!pip install -U spacy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import demoji

# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text

In [4]:
df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/NLP-Social-Media-Mining-for-Health-Monitoring/datasets/Task 2/task3_training.tsv', sep = '\t')
df_eval = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/NLP-Social-Media-Mining-for-Health-Monitoring/datasets/Task 2/task3_validation.tsv', sep = '\t')
df_eval

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver,...",10024668.0,liver damage
2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: ...,10011906.0,death
...,...,...,...,...,...,...,...,...,...
555,331289838856835072,NaN,NaN,NaN,NaN,NaN,"okay trazodone, work your magic.",NaN,NaN
556,332907977700937731,NaN,NaN,NaN,NaN,NaN,"tomorrow, sending mummers her mother's day pos...",NaN,NaN
557,333123450254274560,NaN,NaN,NaN,NaN,NaN,@kyleecakes4 i was given like 5mg of abilify a...,NaN,NaN
558,341950988455927808,NaN,NaN,NaN,NaN,NaN,"stand down, tweeps! stand down! i did it wrong...",NaN,NaN


In [5]:
df = df.dropna(axis=1, how='all') # drop columns that are entirely NAN
df.head()

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,331187619096588288,NaN,NaN,NaN,NaN,ofloxacin,@seefisch:oral drugs for pyelonephritis:ciprof...,NaN,NaN
1,332227554956161024,NaN,NaN,NaN,NaN,trazodone,happy for wellbutrin; has similar effects as a...,NaN,NaN
2,332448217490944000,NaN,NaN,NaN,NaN,lamotrigine,@stilgarg i'm ok ty have an official diagnosis...,NaN,NaN
3,332977955754110976,NaN,NaN,NaN,NaN,cymbalta,i'm soo depressed cymbalta couldn't help me .,NaN,NaN
4,333674203331051520,NaN,NaN,NaN,NaN,seroquel,time for my daily afternoon relaxation ritual ...,NaN,NaN


In [6]:
df_1 = df[df['type'] == "ADR"]
df_0 = df[df['type'] != "ADR"]
df_1

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
27,342314998904786945,42.0,53.0,ADR,lost vision,cymbalta,"#cymbalta withdrawal has reached a peak, lost ...",10047522.0,vision loss
28,342314998904786945,11.0,21.0,ADR,withdrawal,cymbalta,"#cymbalta withdrawal has reached a peak, lost ...",10048010.0,withdrawal syndrome
29,342322703556038657,27.0,35.0,ADR,nauseous,cipro,i hate cipro! #antibiotic #nauseous #cf #hospi...,10028823.0,nauseous
30,342349802601844737,109.0,118.0,ADR,can't cum,seroquel,@luckystubbs reppin zoloft&amp;seroquel since ...,10021574.0,inability to orgasm
31,342349802601844737,101.0,104.0,ADR,fat,seroquel,@luckystubbs reppin zoloft&amp;seroquel since ...,10047896.0,weight gain
...,...,...,...,...,...,...,...,...,...
2239,347414282138431488,38.0,42.0,ADR,flat,zyprexa,"@anyssiakaissa depakote made me feel flat, but...",10016759.0,flat affect
2240,347414282138431488,78.0,89.0,ADR,weight gain,zyprexa,"@anyssiakaissa depakote made me feel flat, but...",10047896.0,weight gain
2242,348383854089871360,72.0,91.0,ADR,gain so much weight,zyprexa,that zyprexa really makes your vocal chords ma...,10047896.0,weight gain
2244,349181683293102080,99.0,117.0,ADR,increase my weight,zyprexa,"i'm so fine today. increasing zyprexa,my condi...",10047898.0,weight increase


In [7]:
def clean_tweet(tweet):
    tweet = re.sub(r"#", "", tweet, 0, re.MULTILINE)
    tweet = re.sub(r"rt", "", tweet, 0, re.MULTILINE)
    # tweet = tweet.lower()
    regex_mentions = r"@\w+"
    tweet = re.sub(regex_mentions, "@USER__", tweet, 0, re.MULTILINE)
#     regex_hash = r"(#[^\s]+)"
#     tweet = re.sub(regex_hash, "", tweet, 0, re.MULTILINE)
#     hash_matched = re.finditer(
#         regex_hash, tweet['full_text'], re.MULTILINE)
    url_regex = r"(http[^\s]+)"
    tweet = re.sub(url_regex, "", tweet, 0, re.MULTILINE)
    # special_char_re = r"[^(?u)\w]+"
    # tweet = re.sub(special_char_re, " ", tweet, 0, re.MULTILINE)
    return tweet

In [8]:
def emoji2Text(text):
    emojiTranslate = { # top 19 emojis used
    '😂': 'laughing',
    '😩': 'distraught',
    '😳': 'surprise',
    '💊': 'medication',
    '😭': 'laughing crying',
    '😒': 'disapproving',
    '😴': 'bored tired',
    '😷': 'sick',
    '😔': 'remorseful',
    '👌': 'correct',
    '👍': 'approve',
    '😫': 'weary',
    '🙌': 'celebration',
    '😏': 'smug',
    '😊': 'happy',
    '😖': 'confounded',
    '💤': 'tired',
    '😁': 'cheerful',
    '❤️': 'love'}

    for word, e in emojiTranslate.items():
        text = text.replace(word.lower(), e)
    
    text = text.lstrip().rstrip()
    
    return text

In [9]:
def remove_emojis(text):
    # emoticons_happy = list([
    #     ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    #     ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    #     '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    #     'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    #     '<3'
    # ])
    # emoticons_sad = list([
    #     ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    #     ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    #     ':c', ':{', '>:\\', ';('
    # ])
    all_emoticons = [':/',':l',':-)','-.-',':::---))))','&lt;',":'(",'-_-',':)','xx','xxx',';-)',
                     ':(',':-/',':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
                     ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D','d:',':-l','&lt;3',
                     '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',':|','o_o',';)',
                     'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',':-0','-__-',':(((',
                     '<3'':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
                     ':-[', ':-<', '=/', '>:(', ':(', '>.<', ":'-(", ":'(",  ':-c',':c', ':{',  ';(', "=\\", ">:\\", ':\\']

    emojis = list(demoji.findall(text).keys())
    clean_text = demoji.replace(text, '')

    for emo in all_emoticons:
        if (emo in clean_text):
            clean_text = clean_text.replace(emo, '')
            emojis.append(emo)

    clean_text = clean_text.replace('\n', '', -1)
    
    # using regex to remove urls
    # regex_url = r"https:(.*)[A-Za-z0-9]"
    # clean_text = re.sub(regex_url, '', clean_text, 0, re.MULTILINE)

    return clean_text #, emojis

In [10]:
# df['tweet'] = df['tweet'].map(lambda tweet: clean_tweet(remove_emojis(emoji2Text(tweet))))
# df_eval['tweet'] = df_eval['tweet'].map(lambda tweet: clean_tweet(remove_emojis(emoji2Text(tweet))))

In [11]:
df

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,331187619096588288,NaN,NaN,NaN,NaN,ofloxacin,@seefisch:oral drugs for pyelonephritis:ciprof...,NaN,NaN
1,332227554956161024,NaN,NaN,NaN,NaN,trazodone,happy for wellbutrin; has similar effects as a...,NaN,NaN
2,332448217490944000,NaN,NaN,NaN,NaN,lamotrigine,@stilgarg i'm ok ty have an official diagnosis...,NaN,NaN
3,332977955754110976,NaN,NaN,NaN,NaN,cymbalta,i'm soo depressed cymbalta couldn't help me .,NaN,NaN
4,333674203331051520,NaN,NaN,NaN,NaN,seroquel,time for my daily afternoon relaxation ritual ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2241,347921687729299456,NaN,NaN,NaN,NaN,NaN,i will admit that most people would agree zypr...,NaN,NaN
2242,348383854089871360,72.0,91.0,ADR,gain so much weight,zyprexa,that zyprexa really makes your vocal chords ma...,10047896.0,weight gain
2243,348595096352075776,NaN,NaN,NaN,NaN,zyprexa,@incubator04 i'm hoping that i get on the zypr...,NaN,NaN
2244,349181683293102080,99.0,117.0,ADR,increase my weight,zyprexa,"i'm so fine today. increasing zyprexa,my condi...",10047898.0,weight increase


In [12]:
df_eval

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver,...",10024668.0,liver damage
2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: ...,10011906.0,death
...,...,...,...,...,...,...,...,...,...
555,331289838856835072,NaN,NaN,NaN,NaN,NaN,"okay trazodone, work your magic.",NaN,NaN
556,332907977700937731,NaN,NaN,NaN,NaN,NaN,"tomorrow, sending mummers her mother's day pos...",NaN,NaN
557,333123450254274560,NaN,NaN,NaN,NaN,NaN,@kyleecakes4 i was given like 5mg of abilify a...,NaN,NaN
558,341950988455927808,NaN,NaN,NaN,NaN,NaN,"stand down, tweeps! stand down! i did it wrong...",NaN,NaN


In [13]:
import spacy


In [17]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [18]:
nlp=spacy.load('en_core_web_sm')
from spacy.training.example import Example

In [19]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [20]:
ner=nlp.get_pipe("ner")

In [21]:
df_ADR = df_1
TRAIN_DATA = []
LABEL = "ADR"
for index, row in df_ADR.iterrows():
    tweet, begin, end = row['tweet'], int(row['begin']), int(row['end'])

    TRAIN_DATA.append((tweet,{'entities':[(begin,end,LABEL)]}))

In [23]:
TRAIN_DATA

[('#cymbalta withdrawal has reached a peak, lost vision and almost crashed my car from a brain zap. thanks a zillion #elililly #bigpharma',
  {'entities': [(42, 53, 'ADR')]}),
 ('#cymbalta withdrawal has reached a peak, lost vision and almost crashed my car from a brain zap. thanks a zillion #elililly #bigpharma',
  {'entities': [(11, 21, 'ADR')]}),
 ('i hate cipro! #antibiotic #nauseous #cf #hospitallife #cysticfibrosis',
  {'entities': [(27, 35, 'ADR')]}),
 ("@luckystubbs reppin zoloft&amp;seroquel since last november. i'm hella gainin weight too awesome i'm fat and can't cum i own",
  {'entities': [(109, 118, 'ADR')]}),
 ("@luckystubbs reppin zoloft&amp;seroquel since last november. i'm hella gainin weight too awesome i'm fat and can't cum i own",
  {'entities': [(101, 104, 'ADR')]}),
 ("@luckystubbs reppin zoloft&amp;seroquel since last november. i'm hella gainin weight too awesome i'm fat and can't cum i own",
  {'entities': [(71, 84, 'ADR')]}),
 ("i've had a trazodone headache al

In [44]:
# for _, annotations in TRAIN_DATA:
#     for ent in annotations.get("entities"):
#         ner.add_label(ent[2])

In [24]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [25]:
# Add the new label to ner
LABEL = "ADR"
ner.add_label(LABEL)

# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [26]:
move_names

['B-ORG',
 'B-DATE',
 'B-PERSON',
 'B-GPE',
 'B-MONEY',
 'B-CARDINAL',
 'B-NORP',
 'B-PERCENT',
 'B-WORK_OF_ART',
 'B-LOC',
 'B-TIME',
 'B-QUANTITY',
 'B-FAC',
 'B-EVENT',
 'B-ORDINAL',
 'B-PRODUCT',
 'B-LAW',
 'B-LANGUAGE',
 'I-ORG',
 'I-DATE',
 'I-PERSON',
 'I-GPE',
 'I-MONEY',
 'I-CARDINAL',
 'I-NORP',
 'I-PERCENT',
 'I-WORK_OF_ART',
 'I-LOC',
 'I-TIME',
 'I-QUANTITY',
 'I-FAC',
 'I-EVENT',
 'I-ORDINAL',
 'I-PRODUCT',
 'I-LAW',
 'I-LANGUAGE',
 'L-ORG',
 'L-DATE',
 'L-PERSON',
 'L-GPE',
 'L-MONEY',
 'L-CARDINAL',
 'L-NORP',
 'L-PERCENT',
 'L-WORK_OF_ART',
 'L-LOC',
 'L-TIME',
 'L-QUANTITY',
 'L-FAC',
 'L-EVENT',
 'L-ORDINAL',
 'L-PRODUCT',
 'L-LAW',
 'L-LANGUAGE',
 'U-ORG',
 'U-DATE',
 'U-PERSON',
 'U-GPE',
 'U-MONEY',
 'U-CARDINAL',
 'U-NORP',
 'U-PERCENT',
 'U-WORK_OF_ART',
 'U-LOC',
 'U-TIME',
 'U-QUANTITY',
 'U-FAC',
 'U-EVENT',
 'U-ORDINAL',
 'U-PRODUCT',
 'U-LAW',
 'U-LANGUAGE',
 'O',
 'B-ADR',
 'I-ADR',
 'L-ADR',
 'U-ADR']

In [27]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path


with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(60):
    
    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        LOSS = []
        texts, annotations = zip(*batch)
        
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
            
        nlp.update(
                    example,
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        LOSS.append(losses['ner'])
        # print("Losses", losses)
    print('maxLoss', max(LOSS), iteration)

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "@thekillmars300 called lamictal (for anxiety) that..." with entities "[(115, 118, 'ADR')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "racing next sun so thought i'd better try on my tr..." with entities "[(105, 122, 'ADR')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030

maxLoss 1795.5412077516323 0
maxLoss 1638.179380597901 1
maxLoss 1536.123179849626 2
maxLoss 1433.6903369903162 3
maxLoss 1377.2935204117068 4
maxLoss 1306.9116370528682 5
maxLoss 1283.3367483776344 6
maxLoss 1258.0091826728512 7
maxLoss 1202.8538526237726 8
maxLoss 1154.727450010543 9
maxLoss 1148.1086747101285 10
maxLoss 1099.3958010936649 11
maxLoss 1044.6087112094642 12
maxLoss 1073.9997598910247 13
maxLoss 1031.584971608652 14
maxLoss 1025.4633348519214 15
maxLoss 995.8503923383015 16
maxLoss 963.3182570347078 17
maxLoss 948.0822329068673 18
maxLoss 938.0438519638684 19
maxLoss 933.1897868037178 20
maxLoss 927.3650880489497 21
maxLoss 889.9343375072964 22
maxLoss 894.7902997708964 23
maxLoss 867.4510262136466 24
maxLoss 868.2956545176982 25
maxLoss 838.5857013456529 26
maxLoss 829.1844463488034 27
maxLoss 836.902475414374 28
maxLoss 849.9013053918411 29
maxLoss 805.7091234563682 30
maxLoss 827.8432952613151 31
maxLoss 794.0073341048441 32
maxLoss 779.4473890358515 33
maxLoss 792.0

In [34]:
test_text = "#eds friends! anybody taken #cipro? (antibiotic) complications?? big side effect is tendon rupture...figured my dr would know better?"
doc = nlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.label_, ent.start_char, ent.end_char)

complications ADR 49 62
tendon rupture ADR 84 98


In [ ]:
for i in range(len(df_eval)):
  row = df.iloc[i]
  if row['type']=='ADR':
    TRAIN_DATA.append((row["tweet"],{'entities':[(int(row['begin']), int(row['end']), "ADR")]}))